# Libs

In [1]:
from  openai import OpenAI # type: ignore
import minsearch # type: ignore
import json
import textwrap
from elasticsearch import Elasticsearch
from elasticsearch import NotFoundError
from tqdm.auto import tqdm

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Api key

In [2]:
apikey = open('txt.txt').read()
client = OpenAI(api_key=apikey)

# Search by prepared search engine

In [3]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
index = minsearch.Index(
        text_fields = ['question', 'text','section'],
        keyword_fields = ['course'],
)
index.fit(documents)

In [10]:
q = 'the course has already started, can I still enroll?'

boost = {'question': 3., 'section': 0.5 }
results = index.search(
            query = q,
            filter_dict = {'course': 'data-engineering-zoomcamp'},
            boost_dict = boost,
            num_results = 5
)
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [11]:
response = client.chat.completions.create(
                model='gpt-4o',
                messages = [{'role':'user', 'content':q}]
)

In [12]:
textwrap.wrap(response.choices[0].message.content, 50)

['The ability to enroll in a course after it has',
 'started depends on several factors, including the',
 'policies of the institution or platform offering',
 "the course, the instructor's discretion, and the",
 "specific course's structure and capacity. Here are",
 "some steps you can take to determine whether it's",
 'possible to enroll late:  1. **Check Enrollment',
 'Policies:**    - Visit the course or institution’s',
 'website to find out if they allow late enrollment.',
 '- Look for specific deadlines for adding courses.',
 '2. **Contact the Instructor:**    - Send an email',
 'or message to the course instructor explaining',
 'your situation and expressing your interest in',
 'joining the course.    - Include any relevant',
 'information, such as why you missed the initial',
 'enrollment period and how you plan to catch up on',
 'missed material.  3. **Contact the Registrar or',
 'Administration:**    - Get in touch with the',
 "registrar's office or the administration of the",

# Gen answer w/ OpenAI + context

In [14]:
prompt_template = """ 
You are a course teaching assistant. 
Answer the QUESTION based on the CONTEXT.
Use only the facts from the CONTEXT when answering
the QUESTION.
If the CONTEXT doesnt contain the answer, output NONE.

QUESTION: {question}

CONTEXT: {context}

""".strip()

context = ''

for doc in results:
    context = (context + 
    f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n")

prompt = prompt_template.format(question=q, context=context).strip()
# print(prompt)

In [15]:
response = client.chat.completions.create(
                model='gpt-4o',
                messages = [{'role':'user', 'content':prompt}]
)
textwrap.wrap(response.choices[0].message.content, 50)

["Yes, even if you don't register, you're still",
 'eligible to submit the homeworks. Be aware,',
 'however, that there will be deadlines for turning',
 "in the final projects. So don't leave everything",
 'for the last minute.']

# The RAG flow

In [16]:
# RAG - retrival augmented generation
def search(query):
    boost = {'question': 3., 'section': 0.5 }
    results = index.search(
                query = query,
                filter_dict = {'course': 'data-engineering-zoomcamp'},
                boost_dict = boost,
                num_results = 10
    )
    return results

def build_prompt(query, search_results):
    prompt_template = """ 
    You are a course teaching assistant. 
    Answer the QUESTION based on the CONTEXT.
    Use only the facts from the CONTEXT when answering
    the QUESTION.
    If the CONTEXT doesnt contain the answer, output NONE.

    QUESTION: {question}

    CONTEXT: {context}

    """.strip()

    context = ''

    for doc in search_results:
        context = (context + 
        f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n")

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
                    model='gpt-4o',
                    messages = [{'role':'user', 'content':prompt}]
    )
    return response.choices[0].message.content

In [17]:
query = 'the course already started. can i enroll?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer
answer = rag(query)
textwrap.wrap(answer,50)

["Yes, even if you don't register, you're still",
 'eligible to submit the homeworks. Be aware,',
 'however, that there will be deadlines for turning',
 "in the final projects, so don't leave everything",
 'for the last minute.']

# Search with Elasticsearch

In [14]:
es_client = Elasticsearch('http://localhost:9200')

In [15]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'course-questions'
es_client.indices.create(index=index_name, body=index_settings)
# try:
#     es_client.indices.get(index=index_name)
#     print(f"{index_name} already exists")
#     # es.indices.delete(index=INDEX_NAME, ignore=[400, 404]) <- uncomment if you want to remove the index.
# except NotFoundError:
#     response = es_client.indices.create(index=index_name, body=index_settings)
#     print(response)

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x74fdbf53e6b0>: Failed to establish a new connection: [Errno 111] Connection refused))

In [7]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document = doc)

  1%|          | 7/948 [00:00<00:28, 32.71it/s]

100%|██████████| 948/948 [00:24<00:00, 39.48it/s]


In [8]:
query = 'I just discovered the course. Can I still join it?'

In [9]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [ ]:
response = es_client.search(index=index_name, body=search_query)

In [ ]:
result_docs = []
for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [ ]:
result_docs